In [14]:
import openeo

In [5]:
eodc_stac = "https://stac.eodc.eu/api/v1"

eodc_openeo = "https://openeo.eodc.eu/v1.0/"
conn = openeo.connect(eodc_openeo).authenticate_oidc(provider_id='egi')

Authenticated using refresh token.


In [6]:
collection      = 'SENTINEL2_L1C'
temporal_extent = ["2021-01-01", "2021-01-15"]

spatial_extent = {
    'west': 11.08348846435547,
    'east': 11.616325378417969,
    'south': 46.44258864468262,
    'north': 46.53713734839792,
    'crs': 'EPSG:4326',
}

bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12']

In [15]:
# This cell is for checking the query exists in the eodc stac
# import pystac_client

# eodc_stac = "https://stac.eodc.eu/api/v1"

# client = pystac_client.Client.open(eodc_stac)

# results = client.search(
#         collections = ["sentinel-2-l1c"],
#         datetime = (temporal_extent[0], temporal_extent[1]),
#         limit = 100,
#         bbox = (spatial_extent['west'], spatial_extent['south'], spatial_extent['north'], spatial_extent['east'])
#     )

# items = list(results.get_items())
# len(items) != 0

In [18]:
cube = conn.load_collection(collection,spatial_extent=spatial_extent,bands=bands,temporal_extent=temporal_extent)
ard = cube.ard_surface_reflectance(atmospheric_correction_method = 'FORCE', cloud_detection_method = 'Fmask', elevation_model = "cop-dem-30m")
ard_tif = ard.save_result(format="netcdf")

In [19]:
job_ard = conn.create_job(ard_tif.flat_graph())
job_id_ard = job_ard.job_id
if job_id_ard:
    print("Batch job created with id: ",job_id_ard)
    job_ard.start_job()
else:
    print("Error! Job ID is None")

Batch job created with id:  jb-8468d93e-9511-474c-99e3-b813588ab77c
